In [1]:
from tqdm import tqdm
from baysic.structure_evaluation import point_energy, relaxed_energy
from baysic.utils import json_to_df
import pandas as pd

No module named 'phonopy'
CHGNet initialized with 400,438 parameters
CHGNet initialized with 400,438 parameters
CHGNet will run on cpu


In [27]:
from mp_api.client import MPRester

with MPRester() as mpr:
    s = mpr.get_structure_by_material_id('mp-11251', conventional_unit_cell=True)

    from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

    sga = SpacegroupAnalyzer(s)
    conv = sga.get_conventional_standard_structure()

    print(conv.composition)
    print(sga.get_crystal_system())

/home/nicholas/anaconda3/envs/baysic/lib/python3.10/site-packages/mp_api/client/mprester.py:227: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Mg6 Au2
hexagonal


In [50]:
from dataclasses import asdict
from baysic.config import SearchConfig
import pyrallis

conf = SearchConfig()
{k: str(v) for k, v in asdict(SearchConfig()).items()}

{'rng_seed': 'None',
 'n_parallel_structs': '500',
 'n_grid': '12',
 'wyckoff_strategy': 'WyckoffSelectionStrategy.WEIGHTED_WP_COUNT',
 'smoke_test': 'False',
 'num_generations': '50',
 'max_gens_at_once': '5',
 'allowed_attempts_per_gen': '5.0',
 'lattice_scale_factor_mu': '1.1',
 'lattice_scale_factor_sigma': '0.47'}

In [61]:
import logging

logging.getLogger().setLevel(logging.CRITICAL)
logging.getLogger().getEffectiveLevel()

50

In [1]:
from baysic.config import MainConfig, TargetStructureConfig, to_dict
conf = MainConfig(target=TargetStructureConfig('mp-66'))

/home/nicholas/anaconda3/envs/baysic/lib/python3.10/site-packages/mp_api/client/mprester.py:227: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


In [3]:
import json
import pyrallis
with pyrallis.config_type('toml'):
    with open('sample_config.toml', 'w') as sample:
        pyrallis.dump(conf, sample)

In [34]:
from pymatgen.core import Composition
Composition('K1N2').to_pretty_string()

'K1N2'

In [38]:
import torch

t = torch.tensor([3, 4, 5, 4, 4, 3, 6, 6, 6, 5]).float()

counts, freqs = t.unique(return_counts=True)
for count, freq in zip(counts, freqs):
    t[t == count] = freq

t

tensor([2., 3., 2., 3., 3., 2., 3., 3., 3., 2.])

In [29]:
symm = sga.get_symmetrized_structure()
symm.wyckoff_symbols

['4f', '2c', '2b']

In [26]:
from dataclasses import asdict, dataclass

@dataclass
class P:
    x: int = 1
    y: int = 2
    
    def __post_init__(self):
        self.z = self.x + self.y

    @property
    def w(self):
        return self.z * 2

p = P()

asdict(p)

{'x': 1, 'y': 2}

In [14]:
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

sga = SpacegroupAnalyzer(s)
conv = sga.get_conventional_standard_structure()

conv.num_sites

10

In [19]:
sga.get_space_group_number()

12

In [2]:
df = pd.read_pickle('merged_test_data3.pkl')
df

,material_id,formula_pretty,nsites,spacegroup,nelements,elements_list,CrystalSystem,category,nontrivial_coordinates,struct
0,mp-557997,CaSeO3,20,14,3,Ca O Se,Monoclinic,polymorph_ternary,14,"[[3.34824742 5.7240056 5.93286188] Ca, [0.887..."
1,mp-13171,YMgCu,9,189,3,Cu Mg Y,Hexagonal,polymorph_ternary,1,"[[-2.18135055 3.7782094 0. ] Y, [ 4...."
2,mp-7550,CeNbO4,12,15,3,Ce Nb O,Monoclinic,polymorph_ternary,7,"[[4.11571443 0.6447703 3.28952249] Ce, [-0.53..."
3,mp-23550,KBrF4,12,140,3,Br F K,Tetragonal,polymorph_ternary,3,"[[1.05567769 1.34280835 2.17287608] K, [ 3.165..."
4,mp-5126,ZnSO4,24,62,3,O S Zn,Orthorhombic,polymorph_ternary,8,"[[2.37637601 3.33392793 4.30100142] Zn, [0. ..."
...,...,...,...,...,...,...,...,...,...,...
175,mp-1106064,Ho4Ga2Ni,17,229,3,Ga Ho Ni,Cubic,template-based_ternary,1,"[[ 2.11805495 2.11805495 -2.11805495] Ho, [ 2..."
176,mp-1105955,Er3Cu3Sb4,20,220,3,Cu Er Sb,Cubic,template-based_ternary,5,"[[-2.3728156 3.55922339 4.74563119] Er, [2...."
177,mp-1105893,La3Cu3Bi4,20,220,3,Bi Cu La,Cubic,template-based_ternary,5,"[[-2.52068287 3.7810243 5.04136574] La, [2...."
178,mp-1105802,CaGe2Pt,16,71,3,Ca Ge Pt,Orthorhombic,template-based_ternary,6,[[5.30951944e-17 2.31557074e+00 6.99969495e-17...


In [3]:
df2 = json_to_df('logs/10-08/Mg6Au2/1/total.json')
df2

,lattice_type,sg_symbol,sg_number,prop_generated,avg_num_successful,prop_actual_success,best_struct,best_relaxed,best_gen_e_form
0,orthorhombic,Immm,71,0.065990,0.129442,1.000000,"[[0. 0. 0.] Mg, [2.04420781 2.94979835 1.96252...",[[7.16965082e-10 1.04241815e-08 6.59768837e-09...,-2.391295
1,triclinic,P-1,2,0.181818,0.927273,0.283333,"[[ 2.56520534 1.1149987 -0.86380184] Mg, [ 0...","[[ 2.54626811 1.01935178 -1.18920356] Mg, [-0...",-2.352042
2,monoclinic,P2,3,0.037037,0.263889,0.500000,"[[3.97847225 0.32643937 0.83354785] Mg, [-3.97...","[[4.21911854 0.30574796 1.11400692] Mg, [-4.21...",-2.347955
3,monoclinic,P21,4,0.411765,3.058824,0.472727,"[[5.54404579 3.24055764 1.08826154] Mg, [-5.54...","[[4.29614811 5.1049023 1.04032217] Mg, [-4.29...",-2.368695
4,monoclinic,C2,5,0.320000,2.160000,0.613636,"[[2.03139127 1.74690874 1.46479703] Mg, [-2.03...","[[1.46842601 1.76491902 2.16507935] Mg, [-1.46...",-2.384298
5,monoclinic,Pm,6,0.094595,0.702703,0.693333,"[[5.61195752 1.49636131 2.40529378] Mg, [ 5.61...","[[7.40604474 1.60670026 2.46954993] Mg, [ 7.40...",-2.319525
6,monoclinic,Pc,7,0.500000,4.214286,0.662921,"[[0.82109824 0.52671466 1.27459706] Mg, [ 1.68...","[[1.31538193 0.59445408 1.80608753] Mg, [ 2.67...",-2.352658
7,monoclinic,Cm,8,0.285714,2.571429,0.627907,"[[5.14480275 0. 3.69206557] Mg, [8.078...",[[ 6.92147457e+00 -1.83601653e-07 4.33006140e...,-2.353315
8,monoclinic,P2/m,10,0.045161,0.335484,0.732394,"[[1.32568439 1.44643582 2.36297214] Mg, [-1.32...","[[1.97237127 1.57864748 2.94482408] Mg, [-1.97...",-2.297317
9,monoclinic,P21/m,11,0.150000,0.900000,0.580645,"[[2.79491711 0. 0. ] Mg, [2.794...","[[ 2.695173 -1.18259489 -0.40836887] Mg, [ 1...",-2.389385


In [6]:
"""Code to generate relaxed structures from values."""
import functools
from math import e
from pymatgen.core import Structure
from scipy.spatial.distance import pdist, squareform
import itertools
import numpy as np

from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from pymatgen.core import Lattice, Structure
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.analysis.molecule_structure_comparator import CovalentRadius

# from matgl import matgl
# from matgl.matgl.ext.ase import M3GNetCalculator, MolecularDynamics, Relaxer
from chgnet.model import StructOptimizer
from chgnet.model.model import CHGNet
import torch

from baysic.utils import upper_tri


MIN_DIST_RATIO = 0.8
VACUUM_SIZE = 7

def is_structure_valid(struct: Structure) -> bool:
    """Tests structure validity."""    
    struct.make_supercell([2, 2, 2], to_unit_cell=False)
    radii = np.array([CovalentRadius.radius[site.specie.symbol] for site in struct.sites])
    # distance threshold
    dists = upper_tri(squareform(pdist(struct.cart_coords)))
    rads = upper_tri(np.add.outer(radii, radii))
    if np.any(dists / rads <= MIN_DIST_RATIO):
        return False
        

    def get_foot(p, a, b):
        p = np.array(p)
        a = np.array(a)
        b = np.array(b)
        ap = p - a
        ab = b - a
        result = a + np.dot(ap, ab) / np.dot(ab, ab) * ab
        return result

    def get_distance(a, b):
        return np.sqrt(np.sum(np.square(b - a)))


    line_a_points = [[0, 0, 0], ]
    line_b_points = [[0, 0, 1], [0, 1, 0], [1, 0, 0],
                        [0, 1, 1], [1, 0, 1], [1, 1, 0], [0, 1, -1], [1, 0, -1], [1, -1, 0],
                        [1, 1, 1], [1, 1, -1], [1, -1, 1], [-1, 1, 1]]
    for a in line_a_points:
        for b in line_b_points:
            foot_points = []
            for p in struct.frac_coords:
                f_p = get_foot(p, a, b)
                foot_points.append(f_p)
            foot_points = sorted(foot_points, key=lambda x: [x[0], x[1], x[2]])

            # 转为笛卡尔坐标
            foot_points = np.asarray(np.mat(foot_points) * np.mat(struct.lattice.matrix))
            for fp_i in range(0, len(foot_points) - 1):
                fp_distance = get_distance(foot_points[fp_i + 1], foot_points[fp_i])
                if fp_distance > VACUUM_SIZE:
                    return False
        
    return True

# eform = matgl.load_model('matgl/pretrained_models/M3GNet-MP-2018.6.1-Eform/model.json')
# pot = matgl.load_model("matgl/pretrained_models/M3GNet-MP-2021.2.8-PES/model.json")
# relaxer = Relaxer(potential=pot)

chgnet = CHGNet.load()
chgnet.to(device='cuda')
relaxer = StructOptimizer()

def point_energy(struct: Structure) -> float:
    prediction = chgnet.predict_structure(struct, task='e')
    if not is_structure_valid(struct):        
        return 100 + prediction['e'].item()
    else: 
        return prediction['e'].item()

def relaxed_energy(struct: Structure, long: bool = False) -> (Structure, float):
    if long:
        params = dict(fmax=0.01, steps=150)    
    else:
        params = dict(fmax=0.02, steps=5)
        
    relax_results = relaxer.relax(struct, **params)
    # extract results
    final_structure = relax_results["final_structure"]
    final_energy = relax_results["trajectory"].energies[-1]
    # print out the final relaxed structure and energy

    return (final_structure, final_energy / final_structure.num_sites)

def e_form(struct: Structure) -> float:    
    if not is_structure_valid(struct):
        return 100
    else:
        return relaxed_energy(struct)[1]

CHGNet initialized with 400,438 parameters
CHGNet initialized with 400,438 parameters
CHGNet will run on cuda:2


In [10]:
import torch

device = 'cuda'
torch.set_default_device(device)

chgnet.to(device=device)

import pandas as pd
from tqdm import tqdm

df = pd.read_pickle('merged_test_data3.pkl')
energies = []

for i in tqdm(df.index[:50]):
    energies.append(point_energy(df.loc[i, 'struct']))

100%|████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 29.88it/s]


- ~14 it/s for Chrom
- ~11 it/s for quantum CPU
- ~10 it/s for pluto CPU

In [4]:
from copy import deepcopy
from tqdm import tqdm

from baysic.structure_evaluation import relaxed_energy
from baysic.utils import df_to_json, json_to_df

df = json_to_df('CeBRh3_raw.json')

best_relaxed = []
best_e_form = []

for struct in tqdm(df['gen']):
    rel, eform = relaxed_energy(deepcopy(struct), long=True)
    best_relaxed.append(rel)
    best_e_form.append(eform)

df['best_relaxed'] = best_relaxed
df['best_e_form'] = best_e_form

df_to_json(df, 'CeBRH3_relaxed.json')

100%|██████████| 1526/1526 [53:34<00:00,  2.11s/it] 


In [6]:
from copy import deepcopy
from tqdm import tqdm

from baysic.structure_evaluation import relaxed_energy
from baysic.utils import df_to_json, json_to_df

df = json_to_df('CeBRh3_raw.json')

best_relaxed = []
best_e_form = []

for struct in tqdm(df['gen']):
    rel, eform = relaxed_energy(deepcopy(struct), long=True)
    best_relaxed.append(rel)
    best_e_form.append(eform)

df['best_relaxed'] = best_relaxed
df['best_e_form'] = best_e_form

df_to_json(df, 'CeBRh3.json')

  0%|                                                                                                                                                                                     | 0/1526 [00:00<?, ?it/s]


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.